In [140]:
from lxml import etree
import lxml.html as LH
from html.parser import HTMLParser
import numpy as np
import pandas as pd
from io import StringIO
import urllib3

In [60]:
# Each results page is for a season and indexed by its end year.
# Choose a year to start then loop back from there until we run out
start_from = 2018

In [61]:
# Teams to gather data for
teams = ['liverpool-fc']

In [62]:
# Connection settings
# http = urllib3.PoolManager()
default_headers = urllib3.make_headers(proxy_basic_auth='shephej:Kjowwnim35')
http = urllib3.ProxyManager("https://10.132.100.135:8080/", headers=default_headers) # 8080

In [63]:
season = start_from
for team in teams:

    # Page to read
    target_page = 'https://www.worldfootball.net/teams/{}/{}/3/'.format(team, season)

    # Read page and save as HTML
    r = http.request('GET', target_page)
    page = r.data.decode('utf-8')

    # Parse the HTML string
    parser = etree.HTMLParser()
    tree = etree.parse(StringIO(page), parser)
    root = tree.getroot()

C:\Users\shephej\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [42]:
<a href="/report/premier-league-1989-1990-liverpool-fc-manchester-city/" title="Match details Liverpool FC - Manchester City">3:1 (1:1) </a>

In [106]:
page_text = root.xpath('//a//text()')
pl_gameweek, opposition, result, date, competition = [], [], [], [], []
for i in range(len(page_text)):
    
    # Get text element
    text = str(page_text[i])
    
    # If we've reached the premier league data, get into it
    if 'Premier League {}/{}'.format(start_from-1, start_from) in text:
        
        # Increment through the data after finding Prem marker
        for j in range(i + 1, len(page_text), 4):  
            prem_text = str(page_text[j])
            
            # First entry of any match should be its round number
            if ' Round' in prem_text:
                pl_gameweek.append(int(prem_text[:2].replace('.', '')))
                
                # If the round number has decreased since last time, that's the end of the Prem
                if len(pl_gameweek) > 1:
                    if pl_gameweek[-1] < pl_gameweek[-2]:
                        break
                    
                # Second entry should be the date
                date.append(str(page_text[j + 1]))

                # Third should be the opposition
                opposition.append(str(page_text[j + 2]))

                # Fourth should be the score
                result.append(str(page_text[j + 3]))
                
                # Record competition
                competition.append('Premier League')
                


In [116]:
fixture_df = pd.DataFrame({'competition': competition,
                          'date': date,
                          'pl_gameweek': pl_gameweek,
                          'team': team,
                          'opposition': opposition,
                          'raw_result': result})
fixture_df['date'] = pd.to_datetime(fixture_df['date'], format='%d/%m/%Y')
fixture_df['team_goals'] = fixture_df['raw_result'].str[0]
fixture_df['opposition_goals'] = fixture_df['raw_result'].str[2]
fixture_df.head()

,competition,date,opposition,pl_gameweek,raw_result,team,team_goals,opposition_goals
0,Premier League,2017-08-12,Watford FC,1,3:3 (1:2),liverpool-fc,3,3
1,Premier League,2017-08-19,Crystal Palace,2,1:0 (0:0),liverpool-fc,1,0
2,Premier League,2017-08-27,Arsenal FC,3,4:0 (2:0),liverpool-fc,4,0
3,Premier League,2017-09-09,Manchester City,4,0:5 (0:2),liverpool-fc,0,5
4,Premier League,2017-09-16,Burnley FC,5,1:1 (1:1),liverpool-fc,1,1


In [126]:
tables = root.xpath('//table')

In [127]:
table = tables[0]

In [131]:
table.prefix

In [135]:
def text(elt):
    return elt.text_content().replace(u'\xa0', u' ')

In [160]:
root = LH.fromstring(page)
for table in root.xpath('//table'):
    header = [text(th) for th in table.xpath('//th')]        # 1
    header = header[:7]
    data = [[text(td) for td in tr.xpath('td')]  
            for tr in table.xpath('//tr')]                   # 2
    #data = [row for row in data if len(row)==len(header)]    # 3 
    #data = pd.DataFrame(data, columns=header)                # 4
    print(data[7])

['Kalenderwoche', '01/08/2017', '19:30', 'A', '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tBayern München\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3:0 (2:0) \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t']
['Kalenderwoche', '01/08/2017', '19:30', 'A', '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tBayern München\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3:0 (2:0) \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t']
['Kalenderwoche', '01/08/2017', '19:30', 'A', '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tBayern München\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t', '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t3:0 (2:0) \r\n\t\t\t\t\t\t\t\t\t\t\t\

TypeError: list indices must be integers or slices, not tuple

In [164]:
header[:7]

['Round', 'date', 'place', '', 'Opponent', 'Results', '']

In [3]:
# Loop through website reads and create a dataframe
for zz in range(len(page_indeces) - 1):
    
    # Print year
    print(zz)
    
    # Page to read
    target_page = 'http://www.lfchistory.net/SeasonArchive/Games/{}'.format(str(page_indeces[zz]))
    
    # Read page and save as HTML
    http = urllib3.PoolManager()
    r = http.request('GET', target_page)
    page = r.data.decode('utf-8')
    
    # Parse the HTML string
    parser = etree.HTMLParser()
    tree = etree.parse(StringIO(page), parser)
    root = tree.getroot()
    
    # Identify data objects I'm interested in
    tds = root.xpath("//td/text()")
    scores = root.xpath("//a/text()")
    scores = scores[1:]
    
    # Initialise lists to fill
    i = 0
    index = []
    date = []
    opposition = []
    venue = []
    competition = []

    # Loop through html data and organise it
    while i < len(tds):

        # Stop when the index resets and brings in friendlies
        if int(tds[i]) < i / 5:
            break

        index.append(tds[i])
        date.append(tds[i+1])
        opposition.append(tds[i+2])
        venue.append(tds[i+3])
        competition.append(tds[i+4])

        i += 5
        
    # Extract the scores
    home_score = []
    away_score = []
    position = 0

    for score in scores:
        home_score.append(score[0])
        away_score.append(score[4])

        # If its not a number then the scores have finished
        try:
            x = int(score[0])
        except ValueError:
            break

        position += 1
        if position >= len(index):
            break
            
    # Create a dataframe of the results
    fixture_history = pd.DataFrame({
        'nth_game_this_season': index,
        'date': date,
        'opposition': opposition,
        'venue': venue,
        'competition': competition,
        'home_score': home_score,
        'away_score': away_score
    })
    
    # Combine results
    if zz == 0:
        final_df = pd.DataFrame({
            'nth_game_this_season': [],
            'date': [],
            'opposition': [],
            'venue': [],
            'competition': [],
            'home_score': [],
            'away_score': []
        })
    else:
        final_df = pd.concat([final_df, fixture_history])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [4]:
final_df['date'] = pd.to_datetime(final_df['date'], format='%d.%m.%Y')

In [5]:
final_df['away_score'] = final_df['away_score'].astype(np.int32)
final_df['home_score'] = final_df['home_score'].astype(np.int32)

In [6]:
import os
out_path = os.path.join(os.pardir, 'data', 'training_data', 'liverpool_fixture_history.csv')
final_df.to_csv(out_path, index=False, encoding='utf-8')